### SentencePiece
découpe les mots. Peut être utilisé en multi-langue (particulièrement pour les langues avec des bases de mots communes)

### WordEmbeddings 
Normalement appliqué après tokenization. Doit être appris sur une langue. Serait-il possible d’apprendre des sentencePice embedding dans plusieurs langue? A tester sur une langue, puis deux, puis 30 (comme MUSE).

### Objectifs : 
* Apprendre sentencePiece
* Apprendre sentencePiece Embeddings
* Comment évaluer les embeddings?
* Apprendre sentencePiece Embeddings en plusieurs langues
* Evaluation?


## Create Embeddings

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
embed_size = 10

In [5]:
word_to_id = {"hello": 0, "world": 1} # create dictionnary of words ids
embeds = nn.Embedding(len(word_to_id), embed_size)

In [6]:
word_tensor = torch.tensor([word_to_id["hello"]], dtype=torch.long)
hello_embed = embeds(word_tensor)
print(hello_embed)

tensor([[ 0.9030,  0.4625,  0.5271, -0.8857,  0.1078,  1.1686, -2.4703,  0.3150,
         -1.5301,  0.5192]], grad_fn=<EmbeddingBackward>)


## Pretrained Embeddings with CBOW

In [9]:
text = """Deep learning (also known as deep structured learning or hierarchical learning) 
is part of a broader family of machine learning methods based on learning data representations, 
as opposed to task-specific algorithms. Learning can be supervised, semi-supervised or unsupervised. 
Deep learning architectures such as deep neural networks, deep belief networks and recurrent neural 
networks have been applied to fields including computer vision, speech recognition, natural language 
processing, audio recognition, social network filtering, machine translation, bioinformatics, 
drug design, medical image analysis, material inspection and board game programs, where they have 
produced results comparable to and in some cases superior to human experts. Deep learning models are 
vaguely inspired by information processing and communication patterns in biological nervous systems 
yet have various differences from the structural and functional properties of biological brains 
(especially human brains), which make them incompatible with neuroscience evidences.""".split()

In [28]:
vocab = set(text)
vocab_size = len(vocab)
word2id = {word:i for i,word in enumerate(vocab)}
id2word = {i:word for i,word in enumerate(vocab)}

### Generate data for training

In [16]:
data = []
for i in range(2, len(text) - 2):
    context = [text[i - 2], text[i - 1],
               text[i + 1], text[i + 2]]
    target = text[i]
    data.append((context, target))
print(data[:5])

[(['Deep', 'learning', 'known', 'as'], '(also'), (['learning', '(also', 'as', 'deep'], 'known'), (['(also', 'known', 'deep', 'structured'], 'as'), (['known', 'as', 'structured', 'learning'], 'deep'), (['as', 'deep', 'learning', 'or'], 'structured')]


In [21]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [26]:
def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]

In [31]:
def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

### CBOW model

In [17]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size=128):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.projection = nn.Sequential(
                            nn.Linear(embedding_dim, hidden_size),
                            nn.ReLU(),
                            nn.Linear(hidden_size, vocab_size),
                            nn.LogSoftmax(dim = -1)
                        )
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.projection(embeds)
        return out

    def get_word_emdedding(self, word):
        word = torch.LongTensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [18]:
model = CBOW(vocab_size, embed_size)

### Learning

In [19]:
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [22]:
for epoch in range(50):
    total_loss = 0
    for context, target in data:
        context_vector = make_context_vector(context, word2id)  
        model.zero_grad()
        log_probs = model(context_vector)
        loss = loss_function(log_probs, torch.tensor([word2id[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.data

### Test

In [37]:
context = ['deep','learning']
context_vector = make_context_vector(context, word2id)
a = model(context_vector).data.numpy()
print('Raw text: {}\n'.format(' '.join(text)))
print('Context: {}\n'.format(context))
print('Prediction: {}'.format(get_max_prob_result(a[0], id2word)))

Raw text: Deep learning (also known as deep structured learning or hierarchical learning) is part of a broader family of machine learning methods based on learning data representations, as opposed to task-specific algorithms. Learning can be supervised, semi-supervised or unsupervised. Deep learning architectures such as deep neural networks, deep belief networks and recurrent neural networks have been applied to fields including computer vision, speech recognition, natural language processing, audio recognition, social network filtering, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases superior to human experts. Deep learning models are vaguely inspired by information processing and communication patterns in biological nervous systems yet have various differences from the structural and functional properties of biological brains (especially human brain